In [1]:
import pandas as pd
import numpy as np

In [2]:
raw_df = pd.read_excel('raw_data.xls', sheet_name='Data', skiprows=3)

In [3]:
indicators = {'SP.POP.TOTL': 'Population', 
              'NY.GDP.MKTP.CD': 'GDP (US$)',
              'SE.XPD.TOTL.GD.ZS': '% Education spending',
              'MS.MIL.XPND.CD': 'Military Spending (US$)',
              'SH.XPD.CHEX.GD.ZS': '% Health spending'
             }



year_data = raw_df.loc[:, ['Country Name', 'Indicator Name', 'Indicator Code', 
                                                                            '2010', '2011', '2012', '2013', '2014', '2015', '2016']]

In [10]:
filtered = year_data.loc[year_data['Indicator Code'].isin(indicators.keys()), :]
filtered.loc[:, 'Indicator Name'] = filtered.loc[:, 'Indicator Code'].apply(lambda x: indicators[x])
filtered.drop(['Indicator Code'], axis = 1, inplace = True)
indexed = filtered.set_index('Country Name')
#indexed.fillna(0, inplace = True)

In [11]:
education = indexed.loc[indexed['Indicator Name'] == '% Education spending', :]
healthcare = indexed.loc[indexed['Indicator Name'] == '% Health spending', :]
gdp = indexed.loc[indexed['Indicator Name'] == 'GDP (US$)', :]
military_total = indexed.loc[indexed['Indicator Name'] == 'Military Spending (US$)', :]
population = indexed.loc[indexed['Indicator Name'] == 'Population']


In [15]:
education.dropna(thresh=7, inplace = True)
education_total = education.copy()
education_total.loc[:, '2010':'2016'] = education.loc[:, '2010':'2016'] * gdp.loc[:, '2010':'2016'] / 100

education_growth = education_total.copy()
education_growth.loc[:, '2010'] = np.NAN
education_growth.loc[:, '2011':'2016'] = education_total.loc[:, '2011':'2016'].values - education_total.loc[:, '2010':'2015'].values
education_growth_per = education_growth.copy()
education_growth_per.loc[:, '2011':'2016'] = education_growth.loc[:, '2011':'2016'].values / education_total.loc[:, '2010':'2015'].values * 100
education_growth_per.drop(['Indicator Name'], axis = 1,  inplace = True)
education_growth.drop(['Indicator Name'], axis = 1,  inplace = True)
education_growth = education_growth.transpose()
education_growth.index.rename('Year', inplace = True)
education_growth_per = education_growth_per.transpose()
education_growth_per.index.rename('Year', inplace = True)

education_total.fillna(0, inplace = True)

healthcare = healthcare.loc[healthcare.index.isin(education.index)]
healthcare_total = healthcare.copy()
healthcare_total.loc[:, '2010':'2016'] = healthcare.loc[:, '2010':'2016'] * gdp.loc[:, '2010':'2016']  / 100

healthcare_growth = healthcare_total.copy()
healthcare_growth.loc[:, '2010'] = np.NAN
healthcare_growth.loc[:, '2011':'2016'] = healthcare_total.loc[:, '2011':'2016'].values - healthcare_total.loc[:, '2010':'2015'].values
healthcare_growth_per = healthcare_growth.copy()
healthcare_growth_per.loc[:, '2011':'2016'] = healthcare_growth.loc[:, '2011':'2016'].values / healthcare_total.loc[:, '2010':'2015'].values * 100
healthcare_growth.drop(['Indicator Name'], axis = 1,  inplace = True)
healthcare_growth = healthcare_growth.transpose()
healthcare_growth.index.rename('Year', inplace = True)
healthcare_growth_per.drop(['Indicator Name'], axis = 1,  inplace = True)
healthcare_growth_per = healthcare_growth_per.transpose()
healthcare_growth_per.index.rename('Year', inplace = True)

military_total = military_total.loc[military_total.index.isin(education.index)]
military_gdp = military_total.copy()
military_gdp.loc[:, '2010':'2016'] = military_total.loc[:, '2010':'2016'] / gdp.loc[:, '2010':'2016'] * 100

population = population.loc[population.index.isin(education.index)]
gdp = gdp.loc[gdp.index.isin(education.index)]
gdp_pcap = gdp.copy()
education_pcap = education_total.copy()
healthcare_pcap = healthcare_total.copy()
military_pcap = military_total.copy()

education_pcap.loc[:,'2010':'2016'] = education_total.loc[:, '2010':'2016'] / population.loc[:, '2010':'2016']
healthcare_pcap.loc[:,'2010':'2016'] = healthcare_total.loc[:, '2010':'2016'] / population.loc[:, '2010':'2016']
military_pcap.loc[:,'2010':'2016'] = military_total.loc[:, '2010':'2016'] / population.loc[:, '2010':'2016']
gdp_pcap.loc[:,'2010':'2016'] = gdp.loc[:, '2010':'2016'] / population.loc[:, '2010':'2016']

shared_education_pcap = education_pcap.copy()
shared_education_pcap.loc[:, '2010':'2016'] = education_pcap.loc[:, '2010':'2016'] / gdp_pcap.loc[:,'2010':'2016']

shared_healthcare_pcap = healthcare_pcap.copy()
shared_healthcare_pcap.loc[:, '2010':'2016'] = healthcare_pcap.loc[:, '2010':'2016'] / gdp_pcap.loc[:,'2010':'2016']

shared_military_pcap = military_pcap.copy()
shared_military_pcap.loc[:, '2010':'2016'] = military_pcap.loc[:, '2010':'2016'] / gdp_pcap.loc[:,'2010':'2016']

excel_file = 'cleaned_data.xlsx'
with pd.ExcelWriter(excel_file, engine='xlsxwriter') as writer: 
    gdp.to_excel(writer, sheet_name = 'GDP')
    gdp_pcap.to_excel(writer, sheet_name = 'GDP_Per_Capita')
    education_total.to_excel(writer, sheet_name = 'Education_Expense')
    education.to_excel(writer, sheet_name = 'Shared_Education')
    education_pcap.to_excel(writer, sheet_name = 'Education_Per_Capita')
    shared_education_pcap.to_excel(writer, sheet_name = 'Shared_Education_PCAP')
    education_growth.to_excel(writer, sheet_name = 'Education_Growth')
    education_growth_per.to_excel(writer, sheet_name = 'Education_Growth_Percentage')
    healthcare_total.to_excel(writer, sheet_name = 'Healthcare_Expense')
    healthcare.to_excel(writer, sheet_name = 'Shared_Healthcare')
    healthcare_pcap.to_excel(writer, sheet_name = 'Healthcare_Per_Capita')
    shared_healthcare_pcap.to_excel(writer, sheet_name = 'Shared_Healthcare_PCAP')
    healthcare_growth.to_excel(writer, sheet_name = 'Healthcare_Growth')
    healthcare_growth_per.to_excel(writer, sheet_name = 'Healthcare_Growth_Percentage')
    military_total.to_excel(writer, sheet_name = 'Military_Expense')
    military_gdp.to_excel(writer, sheet_name = 'Shared_Military')
    military_pcap.to_excel(writer, sheet_name = 'Military_Per_Capita')
    shared_military_pcap.to_excel(writer, sheet_name = 'Shared_Military_PCAP')
    writer.save()



C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
